# Análise Spotify

In [0]:
# Importe das bibliotecas
import pandas as pd
from pyspark.sql.functions import col
import json
%pip install openpyxl

In [0]:
# Leitura dos dados dos arquivos historico extendido
data_2021_2023_audio = pd.read_json("./data/extended/Streaming_History_Audio_2021-2023_0.json")
data_2023_2024_audio = pd.read_json("./data/extended/Streaming_History_Audio_2023-2024_1.json")
data_2024_2025_audio = pd.read_json("./data/extended/Streaming_History_Audio_2024-2025_2.json")
data_2022_2025_video = pd.read_json("./data/extended/Streaming_History_Video_2022-2025.json")

In [0]:
# Leitura dos dados dos arquivos da conta

data_stremingHistoricalMusic = pd.read_json("./data/account/StreamingHistory_music_0.json")
data_stremingHistoricalPodcast = pd.read_json("./data/account/StreamingHistory_podcast_0.json")

In [0]:
# Empinha os dados

data_audio = pd.concat([data_2021_2023_audio, data_2023_2024_audio, data_2024_2025_audio])
# data_video = data_2022_2025_video

display(data_audio.head(5))
# display(data_video.head(5))

In [0]:
# Remove colluns desnecessary

data_audio_clean = data_audio.drop(columns=["conn_country","ip_addr","spotify_episode_uri","audiobook_title","audiobook_uri","audiobook_chapter_uri","audiobook_chapter_title"])
# data_video_clean = data_video.drop(columns=["conn_country","ip_addr",])

In [0]:
# Create 3 columns year, mounth and day from ts column

#Convert ts column to datetime
data_audio_clean['ts'] = pd.to_datetime(data_audio_clean['ts']).dt.tz_localize(None)

#Create columns year
data_audio_clean['year'] = data_audio_clean['ts'].dt.year

#Create columns mounth with zero left

data_audio_clean['mounth'] = data_audio_clean['ts'].dt.strftime('%m')

#Create columns day
data_audio_clean['day'] = data_audio_clean['ts'].dt.strftime('%d')

display(data_audio_clean)

In [0]:
# Freq. Plataform 

display(data_audio_clean['platform'].value_counts())

In [0]:
# cria coluna unica de plataforma

data_audio_clean['platform_clean'] = data_audio_clean['platform'].apply(
    lambda x: (
        "Android" if "android" in x.lower() else
        "Windows" if "windows" in x.lower() else
        "IOS" if "ios" in x.lower() else
        "desconhecido" if "not_applicable" in x.lower() else
        None
    )
)

In [0]:
# Export the cleaned data to Excel files
data_audio_clean.to_excel("./streaming_history_audio_clean.xlsx", index=False)
data_stremingHistoricalMusic.to_excel("./streaming_history_music_clean.xlsx", index=False)
data_stremingHistoricalPodcast.to_excel("./streaming_history_podcast_clean.xlsx", index=False)